In [1]:
import pandas as pd 
from statsmodels.stats.multitest import multipletests
from scipy.stats import wilcoxon

In [ ]:
def pair_test(x):
    '''计算每对基因的pvalue
    '''
    expressionData=[ (float(i.split("-")[0]),float(i.split("-")[1])) for i in x]
    expressionData=pd.DataFrame(expressionData)
    v_,pavl=wilcoxon(
        expressionData[0].values,
        expressionData[1].values,
        alternative='two-sided'
    )
    return pavl

In [ ]:
stage='0DPA'
rawPath='/data/cotton/zhenpingliu/LZP_fiberFullPopulationRNAseq/04homologExpressionPattern/'
BiasGenePairs=pd.read_csv(
    "{}/Homolog_ExpressionBias/{}_has_Bias_geneId.txt".format(
        rawPath,stage
    ),
    header=None,index_col=None,sep="\t"
)


In [ ]:
Stage_ExpressionData=pd.read_csv(
    "{}/Allhomolog_{}_FPKM_V2.txt".format(
        rawPath,stage
    ),
    header=0,index_col=0,sep="\t"
)
#TODO 在至少5%的样本中发生Bias的基因对
BiasGeneList=BiasGenePairs.loc[~BiasGenePairs[1].isin(['BiasN_noExpression','BiasN_Expression'])][0].values
pvalueList=Stage_ExpressionData.loc[BiasGeneList].apply(
    lambda x:pair_test(x),axis=1
)
qvalueList=multipletests(pvalueList.values,method='bonferroni')[1]
#* 合并每个基因 矫正后的结果：
BiasGeneListData=BiasGenePairs.loc[~BiasGenePairs[1].isin(['BiasN_noExpression','BiasN_Expression'])]
BiasGeneListData['pval']=pvalueList.values
BiasGeneListData['qval']=qvalueList

BiasGeneListData.to_csv("./multipleTest/{}_BiasGene_multipleTest.txt".format(stage),header=True,index=False,sep="\t")

In [ ]:
#-----------------------------------------------------
#TODO pvalue矫正换不同的方法
#-----------------------------------------------------
stage='20DPA'
multipleTestBiasGene=pd.read_csv("./multipleTest/{}_BiasGene_multipleTest.txt".format(stage),header=0,index_col=0,sep="\t")
multipleTestBiasGene=multipleTestBiasGene.iloc[:,[0,1,2]]
multipleTestBiasGene.columns=['1', 'pval', 'qval']
qvalueList=multipletests(multipleTestBiasGene['pval'].values,method='fdr_bh')[1] 
multipleTestBiasGene['fdr_bh']=qvalueList
multipleTestBiasGene.to_csv(
    "./multipleTest/{}_BiasGene_multipleTest.txt".format(stage),header=True,index=True,sep="\t"
)

In [ ]:
#---------------------------------------------------------------------
#TODO 所有基因对的Bias类型以及多重校验后的fdr值
#---------------------------------------------------------------------
stage='0DPA'
rawPath='/data/cotton/zhenpingliu/LZP_fiberFullPopulationRNAseq/04homologExpressionPattern/'
BiasGenePairs=pd.read_csv(
    "{}/Homolog_ExpressionBias/{}_has_Bias_geneId.txt".format(
        rawPath,stage
    ),
    header=None,index_col=None,sep="\t"
)
BiasGenePairs.columns=['0','1']
#* 每个时期，多重校验后的结果
multipleTestBiasGene = pd.read_csv(
    "./multipleTest/{}_BiasGene_multipleTest.txt".format(stage), header=0, index_col=None, sep="\t")
MergeData=pd.merge(
    BiasGenePairs,multipleTestBiasGene[['0','1','fdr_bh']],on=['0'],how='outer'
)
MergeData=MergeData[['0','1_x','fdr_bh']]
MergeData.columns=['genePair','BiasType','fdr']
def getBiasType(x):
    geneId,BiasType,fdr=x
    if fdr<=0.05:
        return BiasType
    else:
        return 'BiasN'
MergeData['BiasType_V2']=MergeData.apply(getBiasType,axis=1)
MergeData.to_csv("./multipleTest/{}_homoeologous_fdr.txt".format(stage),header=True,index=False,sep="\t",na_rep=1)

In [ ]:
#* 合并所有时期的结果 
MergeData=pd.DataFrame()
for stage in ['0DPA','4DPA','8DPA','12DPA','16DPA','20DPA']:
    tmpData=pd.read_csv("./multipleTest/{}_homoeologous_fdr.txt".format(stage),header=0,index_col=0,sep="\t")
    MergeData=pd.concat([MergeData,tmpData[['BiasType_V2']]],axis=1)

MergeData.columns= ['0DPA','4DPA','8DPA','12DPA','16DPA','20DPA']
MergeData.to_csv("./multipleTest/All_stageBiastype.txt",header=True,index=True,sep="\t")

In [5]:
#* 所有表达的基因
#* 提取同源基因对中任意一个是表达的状态
expressedGene=pd.read_csv(
    "/public/home/jqyou/out/hg_task/result1_tetroploid/expressed_gene_num/expressed_gene_name_49860.txt",
    header=None,index_col=None,sep="\t"
)
#! 存在Bias的基因
AllHomoeologousData=pd.read_csv("./multipleTest/All_stageBiastype.txt",header=0,index_col=None,sep="\t")

In [17]:
def homoeologousExpressed(homoeologous):
    At,Dt=homoeologous.split("-")
    if expressedGene.loc[expressedGene[0].isin([At,Dt])].empty:
        return 'noExpressed'
    else:
        return 'Expressed'
AllHomoeologousData[
    'expression']=AllHomoeologousData[
    'genePair'
    ].apply(lambda x:homoeologousExpressed(x))   
AllHomoeologousData.to_csv("./multipleTest/All_stageBiastype.txt",header=True,index=False,sep="\t")         

In [30]:
AllHomoeologousData=pd.read_csv("./multipleTest/All_stageBiastype.txt",header=0,index_col=None,sep="\t")

In [119]:
filterBiasGene=AllHomoeologousData.loc[
~(  (AllHomoeologousData['0DPA']==AllHomoeologousData['4DPA'])&(
        AllHomoeologousData['4DPA']==AllHomoeologousData['8DPA']
    )&(AllHomoeologousData['8DPA']==AllHomoeologousData['12DPA'])&(
        AllHomoeologousData['12DPA']==AllHomoeologousData['16DPA']
    )&(AllHomoeologousData['16DPA']==AllHomoeologousData['20DPA'])&(
        AllHomoeologousData['20DPA']=='BiasN'
 ))
]
#* 多重校验且已表达的基因对
filterBiasGene=filterBiasGene.loc[filterBiasGene['expression']=="Expressed"]

In [137]:
AllHomoeologousData.loc[AllHomoeologousData['expression']=='Expressed']['20DPA'].value_counts()

BiasN           9294
BiasD           5433
BiasA           5116
BidirectBias     346
Name: 20DPA, dtype: int64

In [103]:
AllHomoeologousData.loc[
(  (AllHomoeologousData['0DPA']==AllHomoeologousData['4DPA'])&(
        AllHomoeologousData['4DPA']==AllHomoeologousData['8DPA']
    )&(AllHomoeologousData['8DPA']==AllHomoeologousData['12DPA'])&(
        AllHomoeologousData['12DPA']==AllHomoeologousData['16DPA']
    )&(AllHomoeologousData['16DPA']==AllHomoeologousData['20DPA'])&(
        AllHomoeologousData['20DPA']=='BiasA'
 ))
].shape

(1535, 8)

In [63]:
filterBiasGene.loc[filterBiasGene['20DPA']!="BiasN"].shape

(10895, 8)

In [74]:
All_stage_foldChanges=pd.DataFrame()
for stage in ['0DPA','4DPA','8DPA','12DPA','16DPA','20DPA']:
    tmpData=pd.read_csv("./multipleTest/{}_homoeologous_fdr.txt".format(stage),header=0,index_col=0,sep="\t") 
    tmpData=tmpData[['BiasType']]
    All_stage_foldChanges=pd.concat([All_stage_foldChanges,tmpData],axis=1)

In [76]:
def biasType(x):
    if x=='BiasN_Expression':
        return 'BiasN'
    elif x=='BiasN_noExpression':
        return 'BiasN'
    else:
        return x
All_stage_foldChanges=All_stage_foldChanges.applymap(
        lambda x:biasType(x)
    )    
All_stage_foldChanges.columns=['0DPA','4DPA','8DPA','12DPA','16DPA','20DPA']    


In [84]:
All_stage_foldChanges['Homoeologous']=All_stage_foldChanges.index
All_stage_foldChanges[
    'expression']=All_stage_foldChanges['Homoeologous'].apply(lambda x:homoeologousExpressed(x))   

In [87]:
All_stage_Expressed_foldChanges=All_stage_foldChanges.loc[All_stage_foldChanges['expression']=='Expressed']

In [96]:
All_stage_Expressed_foldChanges.loc[
    ~(  (All_stage_Expressed_foldChanges['0DPA']==All_stage_Expressed_foldChanges['4DPA'])&(
        All_stage_Expressed_foldChanges['4DPA']==All_stage_Expressed_foldChanges['8DPA']
    )&(All_stage_Expressed_foldChanges['8DPA']==All_stage_Expressed_foldChanges['12DPA'])&(
        All_stage_Expressed_foldChanges['12DPA']==All_stage_Expressed_foldChanges['16DPA']
    )&(All_stage_Expressed_foldChanges['16DPA']==All_stage_Expressed_foldChanges['20DPA'])&(
        All_stage_Expressed_foldChanges['20DPA']=='BiasN'
 ))
]

,0DPA,4DPA,8DPA,12DPA,16DPA,20DPA,Homoeologous,expression
genePair,,,,,,,,
Ghir_A01G000040-Ghir_D01G000060,BiasA,BiasN,BiasN,BiasN,BiasN,BiasN,Ghir_A01G000040-Ghir_D01G000060,Expressed
Ghir_A01G000540-Ghir_D01G000520,BiasA,BiasN,BiasN,BiasN,BiasN,BiasN,Ghir_A01G000540-Ghir_D01G000520,Expressed
Ghir_A01G000910-Ghir_D01G000870,BiasA,BiasD,BiasD,BiasD,BiasD,BiasD,Ghir_A01G000910-Ghir_D01G000870,Expressed
Ghir_A01G000930-Ghir_D01G000900,BiasA,BiasA,BiasA,BiasA,BiasA,BiasA,Ghir_A01G000930-Ghir_D01G000900,Expressed
Ghir_A01G000940-Ghir_D01G000910,BiasA,BiasA,BiasA,BiasA,BiasN,BiasN,Ghir_A01G000940-Ghir_D01G000910,Expressed
...,...,...,...,...,...,...,...,...
Ghir_A13G024350-Ghir_D13G025080,BiasN,BiasN,BiasD,BiasD,BidirectBias,BiasD,Ghir_A13G024350-Ghir_D13G025080,Expressed
Ghir_A13G024660-Ghir_D13G025430,BiasN,BiasN,BiasD,BiasN,BiasN,BiasD,Ghir_A13G024660-Ghir_D13G025430,Expressed
Ghir_A13G024670-Ghir_D13G025440,BiasN,BiasA,BiasA,BiasD,BiasD,BiasD,Ghir_A13G024670-Ghir_D13G025440,Expressed
